Create a potential plot for a charged disc.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # This import is necessary for 3D plots
from scipy.integrate import dblquad
from ipywidgets import interactive, FloatSlider, IntSlider
from IPython.display import display
from functools import lru_cache
import time

class PotentialCalculator:
    def __init__(self):
        self.k = 9e9
        self.epsilon = 1e-10
        self.cached_potential = None
        self.cached_params = None
        
    @lru_cache(maxsize=1000)
    def single_point_potential_cached(self, q, a, x, y):
        """Cached version of single point potential calculation"""
        sigma = q/a
        x_range = np.sqrt(a/np.pi)
        
        integrand = lambda x_prime, y_prime: sigma/np.sqrt((x-x_prime)**2 + (y-y_prime)**2 + self.epsilon)
        V, _ = dblquad(integrand, -x_range, x_range,
                       lambda x: -x_range,
                       lambda x: x_range)
        return self.k * V

    def calculate_potential_grid(self, q, a, x, y):
        """Calculate potential for entire grid"""
        if self.cached_params == (q, a) and self.cached_potential is not None:
            return self.cached_potential
        
        V = np.zeros_like(x)
        print("Calculating new potential grid...")  # Debug print
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                V[i,j] = self.single_point_potential_cached(q, a, float(x[i,j]), float(y[i,j]))
            if i % 10 == 0:  # Progress indicator
                print(f"Progress: {i}/{x.shape[0]} rows")
        
        self.cached_potential = V
        self.cached_params = (q, a)
        return V

# Create calculator instance
calculator = PotentialCalculator()

# Generate grid once
X = np.linspace(-5, 5, 50)  # Using 30x30 grid for testing
Y = np.linspace(-5, 5, 50)
x, y = np.meshgrid(X, Y)

def potential_plot(elev, azim, q, a):
    start_time = time.time()
    
    # Calculate potential
    Vnet = calculator.calculate_potential_grid(q, a, x, y)
    
    # Clear the current figure
    plt.clf()
    
    # Create new 3D axes - fixed version
    fig = plt.gcf()
    ax = fig.add_subplot(111, projection='3d')
    
    # Create the surface plot
    surf = ax.plot_surface(x, y, Vnet, cmap='magma', edgecolor='none')
    
    # Set the view and style
    ax.view_init(elev=elev, azim=azim)
    ax.dist = 8
    ax.set_box_aspect([1, 1, 0.8])
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    
    # Labels and title
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('Potential (V)')
    
    duration = time.time() - start_time
    ax.set_title(f'3D Potential (Computation time: {duration:.2f}s)', pad=20, y=1.05)
    
    print(f"Plot updated in {duration:.2f} seconds")
    
    plt.draw()

# Create figure with specific size
plt.figure(figsize=(15, 12))

# Create interactive widget
interactive_plot = interactive(
    potential_plot,
    elev=IntSlider(min=-90, max=90, step=5, value=25),
    azim=IntSlider(min=0, max=360, step=5, value=45),
    q=FloatSlider(min=-5, max=5, step=0.1, value=1, description='q'),
    a=FloatSlider(min=0.1, max=10, step=0.1, value=1, description='a')
)

# Display the interactive plot
display(interactive_plot)

<Figure size 1500x1200 with 0 Axes>

interactive(children=(IntSlider(value=25, description='elev', max=90, min=-90, step=5), IntSlider(value=45, de…